# Обучение LSTM на основе предобученного W2V

## Данные

Источник:
https://www.kaggle.com/competitions/sentiment-analysis-in-russian/data?select=train.json

Семантическая классификация на 3 класса.


In [ ]:
!wget -O train.json https://dl.uploadgram.me/637cc99913d70h?raw

--2022-11-22 19:03:29--  https://dl.uploadgram.me/637cc99913d70h?raw
Resolving dl.uploadgram.me (dl.uploadgram.me)... 176.9.247.226, 2a01:4f8:120:30f9:3::1
Connecting to dl.uploadgram.me (dl.uploadgram.me)|176.9.247.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59298269 (57M) [application/json]
Saving to: ‘train.json’

train.json          100%[===================>]  56.55M  4.01MB/s    in 15s     

2022-11-22 19:03:44 (3.89 MB/s) - ‘train.json’ saved [59298269/59298269]



In [ ]:
import pandas as pd

df = pd.read_json('train.json')
df

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative
...,...,...,...
8258,"Как мы писали еще весной, для увеличения сбыта...",10312,positive
8259,Но молодой министр национальной экономики Биши...,10313,negative
8260,\n \nВ ЕНПФ назначен новый председатель правле...,10314,neutral
8261,В Алматы у отделения банка произошло нападение...,10315,negative


Проведем базовую предобработку, уберем пунктуацию и кавычки.

In [ ]:
import string
# реализуем предобработку
def preprocess(doc):
    # к нижнему регистру
    doc = doc.lower()
    # убираем пунктуацию, пробелы, прочее
    for p in string.punctuation + string.whitespace + 'http': 
        doc = doc.replace(p, ' ')
    # убираем кавычки
    for p in ['«', '»', '\'', '\"']:
        doc = doc.replace(p, ' ')
    # убираем лишние пробелы, объединяем обратно
    doc = doc.strip()
    doc = ' '.join([w for w in doc.split(' ') if w != ''])
    return doc

df['text'] = df['text'].map(preprocess)
df

,text,id,sentiment
0,досудебное расследование по факту покупки енпф...,1945,negative
1,медики рассказали о состоянии пострадавшего му...,1957,negative
2,прошел почти год как железнодорожным оператора...,1969,negative
3,по итогам 12 месяцев 2016 года на территории р...,1973,negative
4,астана 21 ноября kazak s an oday агентство рк ...,1975,negative
...,...,...,...
8258,как мы писали еще весной для увеличения сбыта ...,10312,positive
8259,но молодой министр национальной экономики биши...,10313,negative
8260,в енпф назначен новый председатель правления е...,10314,neutral
8261,в алматы у отделения банка произошло нападение...,10315,negative


### Модель из RusVectores

Рассмотрим модель взятую с сайта [RusVectores](https://rusvectores.org/ru/models/):

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/182.zip

--2022-11-22 12:55:56--  http://vectors.nlpl.eu/repository/20/182.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637613799 (608M) [application/zip]
Saving to: ‘182.zip’

182.zip             100%[===================>] 608.08M  51.8MB/s    in 8.5s    

2022-11-22 12:56:05 (71.3 MB/s) - ‘182.zip’ saved [637613799/637613799]



In [ ]:
!unzip 182.zip

Archive:  182.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


Загрузим бинарник в библиотеку генсим:

In [ ]:
import gensim

# Load pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True)

Здесь храняться эмбеддинги каждого слова:

In [ ]:
model.vectors

array([[ 0.10170885,  0.39118028, -0.11257593, ..., -0.26011124,
        -0.22354175, -0.05451489],
       [-0.0765425 ,  0.00272068,  0.13925782, ...,  0.04234103,
         0.14165562, -0.05366017],
       [ 0.06187516,  0.27902287, -0.05537887, ..., -0.10523774,
        -0.1231246 , -0.05266157],
       ...,
       [-0.14584658,  0.11688806, -0.05803058, ..., -0.2306832 ,
         0.04572624, -0.12377718],
       [-0.29234073,  0.03835703,  0.10110392, ..., -0.19030648,
         0.1415805 ,  0.53483826],
       [-0.33604336,  0.01729247, -0.09148607, ..., -0.10110047,
         0.24950679,  0.17947015]], dtype=float32)

In [ ]:
# их форма
model.vectors.shape

In [ ]:
# словарь для каждого слова
model.vocab

{'xxxxxxxx_NUM': <gensim.models.keyedvectors.Vocab at 0x7f2fe1da86d0>,
 'год_NOUN': <gensim.models.keyedvectors.Vocab at 0x7f2fe1da8710>,
 'xxxxxx_NUM': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f350>,
 'xxxxxxx_NUM': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f490>,
 'человек_NOUN': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f250>,
 'время_NOUN': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f4d0>,
 'первый_ADJ': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f550>,
 'один_NUM': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f590>,
 'так_ADV': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f510>,
 'мочь_VERB': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f310>,
 'также_ADV': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f5d0>,
 'быть_VERB': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f610>,
 'район_NOUN': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f650>,
 'другой_ADJ': <gensim.models.keyedvectors.Vocab at 0x7f2fe118f6d0>,
 'город_NOUN': <gensim.models.keyedvect

Установим библиотеку для морфологического анализа. Предобученный w2v слов содержит POS-тег (часть речи).

In [ ]:
!pip install -qq pymorphy2

     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 8.2 MB 9.8 MB/s 


Будем пытаться находить часть речи каждого слова:

In [ ]:
import pymorphy2 as py
morph = py.MorphAnalyzer()

def tag(word):
    try:    
        parsed = morph.parse(word)[0]
        # Падеж .case; выделяется у существительных, полных прилагательных, полных причастий, числительных и местоимений
        return word + '_' + parsed.tag.POS
    except:
        return word

tag('ест')

'ест_VERB'

In [ ]:
len(model.vocab)

248978

In [ ]:
# применим ф-ю нахождение POS тега к каждому слову в предложении
def tag_doc(doc):
    return ' '.join([tag(w) for w in doc.split()])

# применим к каждому предложению
df['text'] = df['text'].map(tag_doc)
df

In [ ]:
df.sentiment.value_counts()

In [ ]:
df.to_csv('preprocessed.csv', index=False)

In [ ]:
# так можно найти индекс слова в словаре
a = model.vocab.get('банка_NOUN')
a.count

22192

Найдем количество пропущенных слов относительно общего числа слов

In [ ]:
missing = 0
total = 0
for i, row in df.iterrows(): # проитерируемся по всему датасету
    for tagword in row['text'].split(): # пройдем по отдельным словам
        if model.vocab.get(tagword) is None: # если слова нет - 
            missing += 1 # добавим к пропущенным
    total += len(row['text'].split()) # добавим общее количество слов предложения

In [ ]:
missing

3721392

In [ ]:
total

4331247

Пропущенно целых 85,5% слов!

In [ ]:
missing / total

0.859196439270261

## Navec


https://github.com/natasha/navec

Преимущества:

1. Работает с торчем
2. Нету шага пос-теггинга
3. Можно подгрузить другие модели из https://rusvectores.org/ru/associates/#
4. Есть возможность квантизовать

Установим библиотеку

In [ ]:
!pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Скачаем основную модель, указанную в гит репозитории:

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2022-11-22 19:38:52--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.3’

navec_hudlit_v1_12B 100%[===================>]  50.56M  47.9MB/s    in 1.1s    

2022-11-22 19:38:53 (47.9 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.3’ saved [53012480/53012480]



Подгрузим полученную модель

In [ ]:
from navec import Navec
navec = Navec.load('/content/navec_hudlit_v1_12B_500K_300d_100q.tar')

Модель содержит вдвое больше слов и не имеет поз тегов, что не требует доп этапов подготовки данных.

In [ ]:
len(navec.vocab.word_ids)

500002

Найдем общее количество пропусков:

In [ ]:
missing = 0
total = 0
for i, row in df.iterrows():
    for word in row['text'].split():
        if navec.vocab.word_ids.get(word) is None:
            missing += 1
    total += len(row['text'].split())

Всего 11%!

In [ ]:
missing / total

0.11594215957547091

In [ ]:
missing

501819

С моделью из торча можно работать как через надстройку slovnet так и через генсим. Генсимовская модель удобна тк позволяет добавить пропущенные слова и передать их в функцию `from_pretrained` слоя `nn.Embedding` .

In [ ]:
gensim_navec = navec.as_gensim
gensim_navec

In [ ]:
len(gensim_navec.vocab)

500002

In [ ]:
# размер вектора эмбеддинга
gensim_navec.vectors[0].shape[0] 

300

In [ ]:
# стандартное отклонение в векторе эмбеддинга
gensim_navec.vectors.std(axis=0).mean()

0.30867122016706344

Для всех уникальных пропущенных слов мы будем создавать эмбеддинг слова (размера 300 в случае модели генсим):

In [ ]:
import numpy as np

np.random.normal(0, 0.3, gensim_navec.vectors[0].shape[0]).shape

(300,)

Найдем все пропущенные слова:

In [ ]:
from tqdm.auto import tqdm
missing_words = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    for word in row['text'].split():
        if navec.vocab.word_ids.get(word) is None:
            missing_words.append(word)
len(missing_words)

  0%|          | 0/8263 [00:00<?, ?it/s]

501819

Посмотрим на количество уникальных, их всего 50к:

In [ ]:
pd.Series(missing_words).value_counts()

–                   15734
—                   15182
1                   12255
2016                10553
2                    8877
                    ...  
кушинге                 1
жильный                 1
гидрометаллургии        1
1228                    1
накален                 1
Length: 50684, dtype: int64

Для всех уникальных слов создадим эмбеддинг из 300 случайных чисел с нулевым матожиданием и дисперсией 0.3, найденной выше. Он будет обучаться вместе с моделью в ходе обработки данных:

In [ ]:
wordList = list(set(missing_words))
vectorList = np.random.normal(0, 0.3, (len(wordList), gensim_navec.vectors[0].shape[0]))
vectorList.shape

(50684, 300)

Добавим новые слова и их эмбеддинги в предобученную генсим модель, полученную от предобученного navec-ом W2V-ка:

In [ ]:
gensim_navec.add(wordList, vectorList)

Суммарно имеем 550к слов с эмбеддингом размера 300:

In [ ]:
gensim_navec.vectors.shape

(550686, 300)

Убедимся в корректности получаемх данных. Эмбеддинги сконвертириованные в пайторч тензор подаются в `nn.Embedding`.

In [ ]:
weights = torch.FloatTensor(gensim_navec.vectors)

In [ ]:
emb = nn.Embedding.from_pretrained(weights, padding_idx = 0)

In [ ]:
gensim_navec['привет'][:10]

array([-0.14197442,  0.01595282, -0.11622857,  0.19758469, -0.54093564,
       -0.23015393,  0.2083324 ,  0.20733036, -0.46802008, -0.43609178])

С помощью это функции мы можем найти индекс слова в словаре. Если его нету в словаре, то мы получим индекс `<unk>` токена. В случае с navec его значение == 50000.

In [ ]:
navec.vocab.get('привет', navec.vocab.unk_id)

335377

Убедимся, что вектор, который выдает Navec для слова "привет" совпадает с тем, что выдает нам nn.Embedding при подаче его индекса:

In [ ]:
emb(torch.tensor([335377], dtype=torch.long))[0][:10]

torch.Size([4, 300])

Очистим из под них память.

In [ ]:
del emb
del weights

### Опциональный подход в работе с Navec

Установим библиотеку slovnet. Он позволяет пользуясь синтаксисом, схожим с `nn.Embedding`, образаться к нужным векторам слов.

In [ ]:
!pip install -qq slovnet

     |████████████████████████████████| 49 kB 3.7 MB/s 


In [ ]:
import torch
from slovnet.model.emb import NavecEmbedding

emb = NavecEmbedding(navec)

sample_text = 'привет как дела фывйцувфывфы'

sample_text = preprocess(sample_text)
print('после предобработки:', sample_text)

indexs = []
for word in sample_text.split():
    # id 500000 == navec.vocab['<unk>'], слово, которого нет в словаре
    # получение индексов слов
    idx = navec.vocab.get(word, navec.vocab.unk_id)
    indexs.append(idx)
    print(word, idx)

input = torch.tensor(indexs) # получаем тензор из индексов
output = emb(input) # получаем эмбеддинги слов (выбор нужных строк из всех эмбеддингов)

after preprocessing: привет как дела фывйцувфывфы
привет 335377
как 161623
дела 104167
фывйцувфывфы 500000


Так выглядит набор векторов, выбранных по индексу слова в словаре, подаваемых в модель:

In [ ]:
output

tensor([[-0.1420,  0.0160, -0.1162,  ...,  0.0409, -0.0816, -0.4607],
        [-0.0204, -0.4305, -0.3130,  ..., -0.0266, -0.1075,  0.5631],
        [-0.3999,  0.1549,  0.0680,  ..., -0.3365, -0.1798,  0.2134],
        [ 0.2143,  0.3703,  0.1368,  ...,  0.2950, -0.0562, -0.1937]])

Пропущенные слова заменяются на эмбеддинг слова `<unk>`

In [ ]:
output.shape # кол-во слов х размер эмбеддинга

torch.Size([4, 300])

In [ ]:
# Индекс пад токена
navec.vocab['<pad>']

500001

In [ ]:
sents = ['это первое предложение', 'это второе предложение фывфывц']

[[navec.vocab.get(word, navec.vocab.unk_id) for word in sent.split()] for sent in sents]

[[496312, 290480, 330951], [496312, 74231, 330951, 500000]]

1. Находим индексы каждого слова. Если слова нет - заменяем индексом слова `<unk>`.
2. Конвертируем в pytorch tensor.
3. Получаем нужные эмбеддинги

In [ ]:
vecs = [emb(torch.tensor([navec.vocab.get(word, navec.vocab.unk_id) for word in sent.split()]))
            for sent in sents]

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# все предложения в батче будут дополняться под длину максимального прдложения
pad_sequence(vecs, padding_value = 500001, batch_first = True)

In [ ]:
# подавая в модель нужно убедиться, что размер эмбеддинга совпадает
lstm = nn.LSTM(input_size = 300, hidden_size = 128, num_layers =1, batch_first =True, )

In [ ]:
# все скрытые и размер cell stat-а
(h_n, c_n) = lstm(pad_sequence(vecs, padding_value = 0, batch_first = True))

In [ ]:
# 2 батча, 4 слова (макс. длина), 128 размер эмбеддинга
h_n.shape

torch.Size([2, 4, 128])

Создадим загрузчик данных:

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Отображение названия класса в номер класса на выходе
num2ans = {
    'neutral': 0,
    'positive': 1,
    'negative':2,
}

class sentiment_dataset(Dataset):
    # сам датасет
    def __init__(self, df):
        self.df = df
    
    # его размер
    def __len__(self):
        return len(self.df)

    # получение примера по индексу
    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0] # предобработанный текст
        label = self.df.iloc[idx, 2] # текст разметки
        label = num2ans[label] # номер класса разметки

        text = torch.tensor([navec.vocab.get(word, navec.vocab.unk_id) for word in text.split()], 
                     dtype=torch.long) # тензор индексов номеров слов в словаре (если нет - unk == 50000)
        return text, label

In [ ]:
train_iter = sentiment_dataset(df)

# функция постобработки батча, дополняет Х под длину самого длинного предложения
def collate_fn(batch):
    x = [e[0] for e in batch] # все индексы слов в словаре
    y = [e[1] for e in batch] # все индексы ответов
    return pad_sequence(x, padding_value = 500001, batch_first = True), torch.tensor(y) # 500001 - пад вектор

# загрузчик
train_dataloader = DataLoader(train_iter, batch_size=30, collate_fn=collate_fn)

In [ ]:
import torch 
from torch.nn.utils.rnn import pad_sequence

# убедимся что все ок загрузилось
next(iter(train_dataloader))

(tensor([[500000, 369832, 302187,  ...,      0,      0,      0],
         [210437, 369655, 252874,  ...,      0,      0,      0],
         [353939, 328321,  91751,  ...,      0,      0,      0],
         ...,
         [ 39474, 161203, 146473,  ...,      0,      0,      0],
         [407271, 163742, 120236,  ...,      0,      0,      0],
         [ 27283, 500000, 103957,  ...,      0,      0,      0]]),
 tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1,
         1, 2, 2, 1, 1, 1]))

In [ ]:
from torch.utils.data import Subset

In [ ]:
# сгенерирует случайные индесы теста
def subset_ind(dataset, ratio: float):
    return np.random.choice(len(dataset), size=int(ratio*len(dataset)), replace=False)

dataset = sentiment_dataset(df)

val_size = 0.2
val_inds = subset_ind(dataset, val_size)

# созаддим train датасет с индексов, не входящих в тестовые
train_dataset = Subset(dataset, [i for i in range(len(dataset)) if i not in val_inds])
# создадим test датасет из инедексов теста
val_dataset = Subset(dataset, val_inds)

In [ ]:
import multiprocessing

# найдем количество ядер для загрузчика
multiprocessing.cpu_count()

2

In [ ]:
# создадим загрузчики для треина и теста
train_dataloader = DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers = multiprocessing.cpu_count())
test_dataloader = DataLoader(val_dataset, batch_size=2, collate_fn=collate_fn)

In [ ]:
import torch.nn as nn

class lstm(nn.Module):
    def __init__(self, w2v, padding_inx, dropout, hidden_size):
        super(lstm, self).__init__()
        # загрузим предобученные эмбеддинги navec-а + 50к не найденных в нем
        self.embedding = nn.Embedding.from_pretrained(w2v)
        self.embedding.padding_inx = padding_inx # зададим индекс паддинга 
        self.embedding.weight.requires_grad = True # веса эмбеддингов будем дообучать вместе с моделью

        self.dropout = nn.Dropout(p = dropout) # регуляризация в процессе обучения
        self.lstm = nn.LSTM(input_size = self.embedding.embedding_dim, # размер эмбеддинга == инпута
                            hidden_size = hidden_size, # размер скрытого представления предложения (эмб.предложения)
                            num_layers = 2, # двуслойный LSTM
                            dropout = dropout, # регуляризация нужна из-за большого размера эмбеддинга
                            bidirectional = True) # каждый слой двунаправленный (т.е. 4 независимых LSTM-a)
        self.label = nn.Linear(hidden_size*2*2, 3) # полносвязный слой подключен к конкатинации 4х последних хидденов

    def forward(self, sentence):
        x = self.embedding(sentence)
        x = torch.transpose(x, dim0 = 1, dim1 = 0) # для подачи в лстм в верном формате
        out, (hidden, c) = self.lstm(x) # прогоняем через все ячейки LSTM
        x = self.dropout(torch.cat([c[i,:,:] for i in range(c.shape[0])], dim=1)) # конкартинируем и регуляризируем
        x = self.label(x) # отображаем для классификации
        return x

In [ ]:
from torch import optim
# 500001 - индекс пад токена в модели Navec
# инициализируем модель
model = lstm(torch.FloatTensor(gensim_navec.vectors), 500001, dropout = 0.2, hidden_size=256)
# и оптимизатор AdamW
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
# функция потерь - кросс-энтропия
loss = nn.CrossEntropyLoss()

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def train(epochs, model, loss_f, optimizer, train_i, val_i):
    step = 0

    losses = []
    val_losses = []

    accuracy = []
    val_accuracy = []

    for epoch in range(epochs):
        for batch in iter(train_i):
            step += 1
            model.train()
            x, y = batch
            # print(x.shape, y.shape)
            model.zero_grad()
            preds = model.forward(x)
            # print(preds)
            # print(preds.shape)
            loss = loss_f(preds, y)
            losses.append(loss.cpu().data.numpy())
            # accuracy.append(f1_score(batch.points.data.numpy().tolist(), 
            #                                np.round(np.array( torch.sigmoid(preds).cpu().data.numpy().tolist()) )) 
            # )
                                           

            loss.backward()
            optimizer.step()
            if step % 50==0:
                print(loss.item())


        clear_output(True)
        model.eval()
        model.zero_grad()
        # val_loss = []

        for batch in iter(val_i):
            x, y = batch

            preds = model.forward(x).view(-1)
            val_losses.append(loss_f(preds, y).cpu().data.numpy())
        
            val_accuracy.append(f1_score(batch.points.data.numpy().tolist(), 
                                            np.round(np.array( torch.sigmoid(preds).cpu().data.numpy().tolist() ))
                                            ))


        fig, axs = plt.subplots(2, 2, figsize=(10, 10))
        fig.suptitle('Accuracy & Loss')
        
        axs[0, 0].set_title('train cross-entropy loss')
        axs[0, 1].set_title('test cross-entropy loss')
        axs[1, 0].set_title('train f1')
        axs[1, 1].set_title('test f1')

        axs[0, 0].plot(losses)
        axs[0, 0].plot(pd.Series(losses).rolling(400).mean().values)
        axs[0, 1].plot(val_losses)
        axs[0, 1].plot(pd.Series(val_losses).rolling(400).mean().values)
        axs[1, 0].plot(accuracy)
        axs[1, 0].plot(pd.Series(accuracy).rolling(400).mean().values)
        axs[1, 1].plot(val_accuracy)
        axs[1, 1].plot(pd.Series(val_accuracy).rolling(400).mean().values)

        for ax in axs.flat:
            ax.set(xlabel='step')
            # axs[0, 0].xaxis.set_ticks(np.arange(0, epochs, 1/len(losses)))
        plt.show()

                # print(f'Эпоха {epoch}, Шаг {step}, train_loss {np.array(losses).mean()}, valid_loss {np.array(val_loss).mean()}')
     

In [ ]:
train(5, model, loss, optimizer, train_dataloader, test_dataloader)

1.1869463920593262
0.8732924461364746


KeyboardInterrupt: ignored